# 8장. 감성 분석에 머신 러닝 적용하기

# 1. 텍스트 처리용 IMDb 영화 리뷰 데이터 준비 

## 영화 리뷰 데이터셋 구하기

압축 푸는 코드

In [6]:
'''
import os
import tarfile

if not os.path.isdir('aclImdb'):

    with tarfile.open('aclImdb_v1.tar.gz', 'r:gz') as tar:
        tar.extractall()
        '''

"\nimport os\nimport tarfile\n\nif not os.path.isdir('aclImdb'):\n\n    with tarfile.open('aclImdb_v1.tar.gz', 'r:gz') as tar:\n        tar.extractall()\n        "

## 영화 리뷰 데이터셋을 더 간편한 형태로 전처리하기

In [7]:
#남은 시간 출력해주는 라이브러리
%pip install pyprind

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


1) 각각의 .txt-> 하나의 CSV로 합침

In [10]:
import pyprind
import pandas as pd
import os

# `basepath`를 압축 해제된 영화 리뷰 데이터셋이 있는 디렉토리로 바꾸세요

basepath = '.\\aclImdb'

labels = {'pos': 1, 'neg': 0} #라벨 붙임
pbar = pyprind.ProgBar(50000)
df = pd.DataFrame()
for s in ('test', 'train'): #test, train 폴더 속
    for l in ('pos', 'neg'):# pos, neg 폴더 속
        path = os.path.join(basepath, s, l) #경로 합침
        for file in sorted(os.listdir(path)):
            with open(os.path.join(path, file), 'r', encoding='utf-8') as infile: #path파일 read로 open
                txt = infile.read()
            df = df.append([[txt, labels[l]]], ignore_index=True) # dateframe 형태로 만듦
            pbar.update()
df.columns = ['review', 'sentiment'] #열 이름 (feature)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:12:18


2. 데이터프레임을 섞음

In [11]:
import numpy as np

np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))

3. 만들어진 데이터를 CSV 파일로 저장

In [12]:
df.to_csv('movie_data.csv', index=False, encoding='utf-8')

4. csv파일 출력해봄(test차원)

In [2]:
import pandas as pd

df = pd.read_csv('movie_data.csv', encoding='utf-8')
df.head(3)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0


In [4]:
#행과 열 체크 50000개 샘플, 2개 열(review, sentiment)
df.shape

(50000, 2)

# 2. BoW 모델 소개

### 1) 단어를 특성 벡터로 변환하기

CountVectorizer의 fit_transform 메서드를 호출하여 BoW 모델의 어휘사전을 만들고, 세 문장을 희소한 특성 벡터로 변환합니다:


In [5]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer() #사전 생성

#희소한 특성 벡터로 변환
docs = np.array([
        'The sun is shining',
        'The weather is sweet',
        'The sun is shining, the weather is sweet, and one and one is two'])
bag = count.fit_transform(docs)

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


어휘 사전 출력

In [6]:
print(count.vocabulary_) #딕셔너리에 저장됨 {단어:정수 인덱스}

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}


특성 벡터 출력
- 딕셔너리의 정수값 = 특성벡터의 인덱스// 행=문장 번호, 열=인덱스, 값=단어 빈도(term frequency)
  - ex> 'and': 0 // 3번행 0번열=> 값2  ==> 3번째 문장에 and가 2번 등장했다

문서 d에 등장한 단어 t의 횟수 = tf (t,d)

In [7]:
print(bag.toarray())

[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


### 2)  사이킷런 tf-idf를 사용해 단어 적합성 평가하기

특성 벡터에서 자주 등장하는 단어의 가중치를 낮춤

<표준공식><br><br>
tf-idf 
$$\text{tf-idf}(t,d)=\text{tf (t,d)}\times \text{idf}(t,d)$$
- tf(t, d)=단어 빈도
- idf(t, d)=역문서 빈도
- 이 전에 단어 빈도 정규화 해줌

역문서 빈도

$$\text{idf}(t,d) = \text{log}\frac{n_d}{1+\text{df}(d, t)},$$

- $n_d$= 전체 문서 개수
- df(d, t)= 단어 t가 포함된 문서 d의 개수
- 분모에 상수 1을 추가하는 것은 선택 (훈련 샘플에 한 번도 등장하지 않는 단어가 있는 경우 분모가 0이 되지 않게 만듬)
- log는 문서 빈도 df(d, t)가 낮을 때 역문서 빈도 값이 너무 커지지 않도록 조절

#### 2-1) 사이킷런 라이브러리 `TfidfTransformer`
- 표준공식과는 조금 다름
- L2규제 사용


결과: 빈도가 가장 높았던 is-> 작은 tf-idf (0.45)를 갖음-> 유용한 정보 아님

In [8]:
np.set_printoptions(precision=2)

In [9]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer(use_idf=True, 
                         norm='l2', 
                         smooth_idf=True)

print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[0.   0.43 0.   0.56 0.56 0.   0.43 0.   0.  ]
 [0.   0.43 0.   0.   0.   0.56 0.43 0.   0.56]
 [0.5  0.45 0.5  0.19 0.19 0.19 0.3  0.25 0.19]]


#### 2-2) 직접 구현

<사이킷런 공식><br><br>
tf-idf
$$\text{tf-idf}(t,d) = \text{tf}(t,d) \times (\text{idf}(t,d)+1)$$
- tf(t, d)=단어 빈도
- idf(t, d)=역문서 빈도
- tf-idf를 직접 정규화함(l2)
$$v_{\text{norm}} = \frac{v}{||v||_2} = \frac{v}{\sqrt{v_{1}^{2} + v_{2}^{2} + \dots + v_{n}^{2}}} = \frac{v}{\big (\sum_{i=1}^{n} v_{i}^{2}\big)^\frac{1}{2}}$$

역문서 빈도

$$\text{idf} (t,d) = log\frac{1 + n_d}{1 + \text{df}(d, t)}$$

- $n_d$= 전체 문서 개수
- df(d, t)= 단어 t가 포함된 문서 d의 개수
- 분모에 상수 1을 추가하는 것은 선택 (훈련 샘플에 한 번도 등장하지 않는 단어가 있는 경우 분모가 0이 되지 않게 만듬)
- log는 문서 빈도 df(d, t)가 낮을 때 역문서 빈도 값이 너무 커지지 않도록 조절

ex> 세번째 문서의 ‘is'의 tf-idf를 계산

세 번째 문서에서 단어 ‘is’의 단어 빈도는 3(tf=3). 이 단어는 세 개 문서에 모두 나타나기 때문에 문서 빈도가 3(df=3). 

역문서 빈도 
$$\text{idf}("is", d3) = log \frac{1+3}{1+3} = 0$$

tf-idf

$$\text{tf-idf}("is",d3)= 3 \times (0+1) = 3$$

In [10]:
tf_is = 3
n_docs = 3
idf_is = np.log((n_docs+1) / (3+1))
tfidf_is = tf_is * (idf_is + 1)
print('tf-idf of term "is" = %.2f' % tfidf_is)

tf-idf of term "is" = 3.00


세 번째 문서에 있는 모든 단어에 대해 이런 계산을 반복 tf-idf 벡터 [3.39, 3.0, 3.39, 1.29, 1.29, 1.29, 2.0, 1.69, 1.29]를 얻음
tf-idf 계산의 마지막-> L2-정규화

$$\text{tfi-df}_{norm} = \frac{[3.39, 3.0, 3.39, 1.29, 1.29, 1.29, 2.0 , 1.69, 1.29]}{\sqrt{[3.39^2, 3.0^2, 3.39^2, 1.29^2, 1.29^2, 1.29^2, 2.0^2 , 1.69^2, 1.29^2]}}$$

$$=[0.5, 0.45, 0.5, 0.19, 0.19, 0.19, 0.3, 0.25, 0.19]$$

$$\Rightarrow \text{tfi-df}_{norm}("is", d3) = 0.45$$

사이킷런의 `TfidfTransformer`에서 반환된 결과와 같아짐

In [11]:
tfidf = TfidfTransformer(use_idf=True, norm=None, smooth_idf=True)
raw_tfidf = tfidf.fit_transform(count.fit_transform(docs)).toarray()[-1]
raw_tfidf 

array([3.39, 3.  , 3.39, 1.29, 1.29, 1.29, 2.  , 1.69, 1.29])

In [12]:
#l2 정규화
l2_tfidf = raw_tfidf / np.sqrt(np.sum(raw_tfidf**2))
l2_tfidf

array([0.5 , 0.45, 0.5 , 0.19, 0.19, 0.19, 0.3 , 0.25, 0.19])

### 3) 텍스트 데이터 정제

In [13]:
#마지막 50개를 출력 -> html문법도 섞여 있음
df.loc[0, 'review'][-50:]

'is seven.<br /><br />Title (Brazil): Not Available'

정규식으로 제거

In [14]:
import re
def preprocessor(text):
    #html 마크업 삭제(parsing)
    text = re.sub('<[^>]*>', '', text)
    #이모티콘 찾음
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',text)
    #소문자 변환->[/w] 단어가 아닌 문자 제거 ->emoticon변수를 처리가 완료된 문자열 끝에 추가 + - 기호 삭제
    text = (re.sub('[\W]+', ' ', text.lower()) +
' '.join(emoticons).replace('-', ''))
    return text

In [15]:
preprocessor(df.loc[0, 'review'][-50:])

'is seven title brazil not available'

In [16]:
preprocessor("</a>This :) is :( a test :-)!")

'this is a test :) :( :)'

모든 데이터 프레임에 함수 적용

In [17]:
df['review'] = df['review'].apply(preprocessor)

In [18]:
df['review'].map(preprocessor)

0        in 1974 the teenager martha moxley maggie grac...
1        ok so i really like kris kristofferson and his...
2         spoiler do not read this if you think about w...
3        hi for all the people who have seen this wonde...
4        i recently bought the dvd forgetting just how ...
                               ...                        
49995    ok lets start with the best the building altho...
49996    the british heritage film industry is out of c...
49997    i don t even know where to begin on this one i...
49998    richard tyler is a little boy who is scared of...
49999    i waited long to watch this movie also because...
Name: review, Length: 50000, dtype: object

### 4) 문서를 토큰으로 나누기

NLTK에 구현된 포터 어간 추출기

In [19]:
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()

#띄어쓰기 기반 분류
def tokenizer(text):
    return text.split()

#포터 어간 추출 분류
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [20]:
tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

In [21]:
tokenizer_porter('runners like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

불용어 제거

In [22]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jsm36\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [23]:
from nltk.corpus import stopwords

stop = stopwords.words('english')
[w for w in tokenizer_porter('a runner likes running and runs a lot')[-10:]
if w not in stop]

['runner', 'like', 'run', 'run', 'lot']

# 3. 문서 분류를 위한 로지스틱 회귀 모델 훈련하기

### 1) train/ test 분리

In [24]:
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

### 2) 5-겹 계층별 교차 검증을 사용 -> 최적의 매개변수 조합 찾음

In [25]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV

#
tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)

#시도 해볼 매개 변수의 조합
param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              {'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'vect__use_idf':[False],
               'vect__norm':[None],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              ]

#로지스틱 회귀 파이프라인
lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(solver='liblinear', random_state=0))])

#로지스틱 회귀모델을 다양한 매개변수의 조합 훈련
gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5,
                           verbose=1,
                           n_jobs=1)

 모든 CPU 코어를 사용해 그리드 서치의 속도를 높이려면 (`n_jobs=1` 대신) `n_jobs=-1`로 지정하는 것이 좋다. 일부 시스템에서는 멀티프로세싱을 위해 `n_jobs=-1`로 지정할 때 `tokenizer` 와 `tokenizer_porter` 함수의 직렬화에 문제가 발생할 수 있다. 이런 경우 `[tokenizer, tokenizer_porter]`를 `[str.split]`로 바꾸어 문제를 해결할 수 있습니다. 다만 `str.split`로 바꾸면 어간 추출을 하지 못한다.

매개변수 그리드에서 정의한 대로 2*2*2*3*5 + 2*2*2*3*5 = 240개의 모델을 훈련


<학습 속도를 높이는 밥> => 성능 하락
- 훈련 세트 크기를 줄임
- 그리드에 지정한 매개변수를 삭제


### 3) 학습

In [26]:
gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


In [ ]:
print('최적의 매개변수 조합: %s ' % gs_lr_tfidf.best_params_)
print('CV 정확도: %.3f' % gs_lr_tfidf.best_score_)

최적의 매개변수 조합: {'clf__C': 10.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer at 0x7f4387eb0950>} 
CV 정확도: 0.897


In [ ]:
clf = gs_lr_tfidf.best_estimator_
print('테스트 정확도: %.3f' % clf.score(X_test, y_test))

테스트 정확도: 0.899


# 4. 대용량 데이터 처리-온라인 알고리즘과 외부 메모리 학습

In [ ]:
# 이 셀의 코드는 책에 포함되어 있지 않습니다. This cell is not contained in the book but
# 이전 코드를 실행하지 않고 바로 시작할 수 있도록 편의를 위해 추가했습니다.

import os
import gzip


if not os.path.isfile('movie_data.csv'):
    if not os.path.isfile('movie_data.csv.gz'):
        print('Please place a copy of the movie_data.csv.gz'
              'in this directory. You can obtain it by'
              'a) executing the code in the beginning of this'
              'notebook or b) by downloading it from GitHub:'
              'https://github.com/rasbt/python-machine-learning-'
              'book-2nd-edition/blob/master/code/ch08/movie_data.csv.gz')
    else:
        in_f = gzip.open('movie_data.csv.gz', 'rb')
        out_f = open('movie_data.csv', 'wb')
        out_f.write(in_f.read())
        in_f.close()
        out_f.close()

In [ ]:
import numpy as np
import re
from nltk.corpus import stopwords


# `stop` 객체를 앞에서 정의했지만 이전 코드를 실행하지 않고
# 편의상 여기에서부터 코드를 실행하기 위해 다시 만듭니다.
stop = stopwords.words('english')


def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized


def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv)  # 헤더 넘기기
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

In [ ]:
next(stream_docs(path='movie_data.csv'))

('"In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />""Murder in Greenwich"" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich f

In [ ]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        pass
    return docs, y

In [ ]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier


vect = HashingVectorizer(decode_error='ignore', 
                         n_features=2**21,
                         preprocessor=None, 
                         tokenizer=tokenizer)

In [ ]:
from distutils.version import LooseVersion as Version
from sklearn import __version__ as sklearn_version

clf = SGDClassifier(loss='log', random_state=1, max_iter=1)

doc_stream = stream_docs(path='movie_data.csv')

In [ ]:
import pyprind
pbar = pyprind.ProgBar(45)

classes = np.array([0, 1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

/home/haesun/anaconda3/envs/python-ml/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:18


In [ ]:
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print('정확도: %.3f' % clf.score(X_test, y_test))

정확도: 0.867


In [ ]:
clf = clf.partial_fit(X_test, y_test)

## 토픽 모델링

### 사이킷런의 LDA

In [ ]:
import pandas as pd

df = pd.read_csv('movie_data.csv', encoding='utf-8')
df.head(3)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english',
                        max_df=.1,
                        max_features=5000)
X = count.fit_transform(df['review'].values)

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components=10,
                                random_state=123,
                                learning_method='batch')
X_topics = lda.fit_transform(X)

In [ ]:
lda.components_.shape

(10, 5000)

In [ ]:
n_top_words = 5
feature_names = count.get_feature_names()

for topic_idx, topic in enumerate(lda.components_):
    print("토픽 %d:" % (topic_idx + 1))
    print(" ".join([feature_names[i]
                    for i in topic.argsort()\
                        [:-n_top_words - 1:-1]]))

토픽 1:
worst minutes awful script stupid
토픽 2:
family mother father children girl
토픽 3:
american war dvd music tv
토픽 4:
human audience cinema art sense
토픽 5:
police guy car dead murder
토픽 6:
horror house sex girl woman
토픽 7:
role performance comedy actor performances
토픽 8:
series episode war episodes tv
토픽 9:
book version original read novel
토픽 10:
action fight guy guys cool


각 토픽에서 가장 중요한 단어 다섯 개를 기반으로 LDA가 다음 토픽을 구별했다고 추측할 수 있습니다.

1.	대체적으로 형편없는 영화(실제 토픽 카테고리가 되지 못함)
2.	가족 영화
3.	전쟁 영화
4.	예술 영화
5.	범죄 영화
6.	공포 영화
7.	코미디 영화
8.	TV 쇼와 관련된 영화
9.	소설을 원작으로 한 영화
10.	액션 영화

카테고리가 잘 선택됐는지 확인하기 위해 공포 영화 카테고리에서 3개 영화의 리뷰를 출력해 보죠(공포 영화는 카테고리 6이므로 인덱스는 5입니다):

In [ ]:
horror = X_topics[:, 5].argsort()[::-1]

for iter_idx, movie_idx in enumerate(horror[:3]):
    print('\n공포 영화 #%d:' % (iter_idx + 1))
    print(df['review'][movie_idx][:300], '...')


공포 영화 #1:
House of Dracula works from the same basic premise as House of Frankenstein from the year before; namely that Universal's three most famous monsters; Dracula, Frankenstein's Monster and The Wolf Man are appearing in the movie together. Naturally, the film is rather messy therefore, but the fact that ...

공포 영화 #2:
Okay, what the hell kind of TRASH have I been watching now? "The Witches' Mountain" has got to be one of the most incoherent and insane Spanish exploitation flicks ever and yet, at the same time, it's also strangely compelling. There's absolutely nothing that makes sense here and I even doubt there  ...

공포 영화 #3:
<br /><br />Horror movie time, Japanese style. Uzumaki/Spiral was a total freakfest from start to finish. A fun freakfest at that, but at times it was a tad too reliant on kitsch rather than the horror. The story is difficult to summarize succinctly: a carefree, normal teenage girl starts coming fac ...


앞 코드에서 공포 영화 카테고리 중 최상위 3개의 리뷰에서 300자씩 출력했습니다. 정확히 어떤 영화에 속한 리뷰인지는 모르지만 공포 영화의 리뷰임을 알 수 있습니다.